# Analysis
Weigh the odds against the predictions and make your bet!

The selected bets are stored in `./Book.csv`.

Remember to fill in the actualy results of your bet in the `Book` after the match.

In [1]:
import pandas as pd

## Read the odds and predictions files

In [2]:
# odds
dfOdds = pd.read_csv('./data/This_months_odds.csv')
print(dfOdds.shape)
display(dfOdds.head())

# predictions
dfPreds = pd.read_csv('./data/This_months_predictions.csv')
print(dfPreds.shape)
dfPreds.head()

(79, 6)


,away_team,home_team,odd_away_win,odd_home_win,odd_tie,league
0,ado den haag,willem ii,3.75,1.95,3.6,eredivisie
1,nac breda,pec zwolle,5.50,1.60,4.0,eredivisie
2,excelsior rotterdam,az alkmaar,9.00,1.30,5.5,eredivisie
3,heracles almelo,fc utrecht,5.40,1.60,4.5,eredivisie
4,fc groningen,vvv venlo,2.25,3.20,3.6,eredivisie


(186, 7)


,date,league,home_team,away_team,prob_home_win,prob_away_win,prob_tie
0,2018-12-09,barclays premier league,newcastle,wolverhampton,0.4229,0.2800,0.2971
1,2018-12-10,barclays premier league,everton,watford,0.4477,0.2740,0.2783
2,2018-12-15,barclays premier league,manchester city,everton,0.8786,0.0249,0.0965
3,2018-12-15,barclays premier league,huddersfield town,newcastle,0.4009,0.2788,0.3203
4,2018-12-15,barclays premier league,watford,cardiff city,0.5195,0.2146,0.2659


## Unpivot to go from matches to bets
So each row in the table corresponds to 1 possible bet (home win, away win or tie).

In [3]:
# unpivot the odds columns
dfOdds = dfOdds.melt(id_vars=['league', 'home_team', 'away_team'])

# strip "odd_" from the bet outcome
dfOdds.variable = dfOdds.variable.str.replace('odd_', '')

# rename the column we got from the unpivot (melt)
dfOdds.rename(columns={'variable': 'outcome', 'value': 'odd'}, inplace=True)

# show
print(dfOdds.shape)
dfOdds.head()

(237, 5)


,league,home_team,away_team,outcome,odd
0,eredivisie,willem ii,ado den haag,away_win,3.75
1,eredivisie,pec zwolle,nac breda,away_win,5.50
2,eredivisie,az alkmaar,excelsior rotterdam,away_win,9.00
3,eredivisie,fc utrecht,heracles almelo,away_win,5.40
4,eredivisie,vvv venlo,fc groningen,away_win,2.25


In [4]:
# unpivot the prediction columns
dfPreds = dfPreds.melt(id_vars=['date', 'league', 'home_team', 'away_team'])
dfPreds

# strip "prob_" from the bet outcome
dfPreds.variable = dfPreds.variable.str.replace('prob_', '')

# rename the column we got from the unpivot (melt)
dfPreds.rename(columns={'variable': 'outcome', 'value': 'prob'}, inplace=True)

# show
print(dfPreds.shape)
dfPreds.head()

(558, 6)


,date,league,home_team,away_team,outcome,prob
0,2018-12-09,barclays premier league,newcastle,wolverhampton,home_win,0.4229
1,2018-12-10,barclays premier league,everton,watford,home_win,0.4477
2,2018-12-15,barclays premier league,manchester city,everton,home_win,0.8786
3,2018-12-15,barclays premier league,huddersfield town,newcastle,home_win,0.4009
4,2018-12-15,barclays premier league,watford,cardiff city,home_win,0.5195


## Do some data mapping to align both tables

In [5]:
# map the league names to match the odds
leaguesMap = {'french ligue 1': 'ligue_1',
              'german bundesliga': 'bundesliga',
              'dutch eredivisie': 'eredivisie',
              'italy serie a': 'serie_a',
              'spanish primera division': 'la_liga',
              'barclays premier league': 'premier_league'}
dfPreds['league'] = dfPreds.league.replace(leaguesMap)

In [6]:
# check if team names differ from odds / preds
# all team names from odds (filter 1 league)
filt = dfOdds.league == 'ligue_1'
teamsFromOdds = set(dfOdds[filt].home_team).union(set(dfOdds[filt].away_team))

# all teams names from predictions (filter 1 league)
filt = dfPreds.league == 'ligue_1'
teamsFromPreds = set(dfPreds[filt].home_team).union(set(dfPreds[filt].away_team))

# find set difference
teamsFromOdds.symmetric_difference(teamsFromPreds)

{'as monaco',
 'dijon',
 'dijon fco',
 'monaco',
 'nimes',
 'nimes olympique',
 'paris saint-germain',
 'paris sg',
 'rennes',
 'saint-etienne',
 'st etienne',
 'stade rennes'}

In [7]:
# map the team names to match the odds
teamsMap = { 
    # NL
    'emmen': 'fc emmen',
    'nac': 'nac breda',
    'heerenveen': 'sc heerenveen',
    'excelsior': 'excelsior rotterdam',
    'psv': 'psv eindhoven',
    'az': 'az alkmaar',
    'heracles': 'heracles almelo',
    # ES
    'athletic bilbao': 'athletic club bilbao',
    'sevilla fc': 'sevilla',
    'real valladolid': 'valladolid',
    'sd huesca': 'huesca',
    'barcelona': 'fc barcelona',
    # DE
    'mainz': 'mainz 05',
    'tsg hoffenheim': 'hoffenheim',
    'fc augsburg': 'augsburg',
    'bayern munich': 'bayern munchen',
    # EN
    'afc bournemouth': 'bournemouth',
    'west ham united': 'west ham',
    'tottenham hotspur': 'tottenham',
    'brighton and hove albion': 'brighton & hove albion',
    'newcastle': 'newcastle united',
    'wolverhampton': 'wolverhampton wanderers',
    # IT
    'as roma': 'roma',
    'internazionale': 'inter',
    # FR
    'paris saint-germain': 'paris sg',
    'st etienne': 'saint-etienne',
    'dijon fco': 'dijon',
    'nimes': 'nimes olympique',
    'stade rennes': 'rennes',
    'as monaco': 'monaco'
}
dfPreds['home_team'] = dfPreds.home_team.replace(teamsMap)
dfPreds['away_team'] = dfPreds.away_team.replace(teamsMap)

In [8]:
# try to merge on [league], [home_team] and [away_team]
dfJoin = dfOdds.merge(dfPreds, how='left', indicator=True,
                      on=['league', 'home_team', 'away_team', 'outcome']
                     )

# check for mismatches - result should be empty data frame
display(dfJoin[dfJoin['_merge'] == 'left_only'])

# and then drop _merge to continue
dfJoin.drop(columns=['_merge'], inplace=True)

,league,home_team,away_team,outcome,odd,date,prob,_merge


## Calculate expectation values and select bets
Expectation value = odd * probability

Select bets with `expect` > 1.15

In [9]:
# calculate the expectation value for each possible bet
dfJoin['expect'] = dfJoin.odd * dfJoin.prob
print(dfJoin.shape)

(237, 8)


In [10]:
# select bets higher then the threshold
dfBets = dfJoin[dfJoin.expect > 1.15].copy()

# sort and set a fixed column order
columnOrder = ['date', 'league', 'home_team', 'away_team', 'outcome', 'odd', 'prob', 'expect']
dfBets = dfBets.sort_values(by=['league', 'date'])[columnOrder]

# show
print(dfBets.shape)
dfBets

(26, 8)


,date,league,home_team,away_team,outcome,odd,prob,expect
108,2018-12-14,bundesliga,1. fc nurnberg,vfl wolfsburg,home_win,3.45,0.3630,1.252350
117,2018-12-15,bundesliga,augsburg,schalke 04,home_win,2.85,0.4045,1.152825
115,2018-12-16,bundesliga,eintracht frankfurt,bayer leverkusen,home_win,2.35,0.5051,1.186985
3,2018-12-09,eredivisie,fc utrecht,heracles almelo,away_win,5.40,0.2302,1.243080
83,2018-12-09,eredivisie,vvv venlo,fc groningen,home_win,3.20,0.3790,1.212800
90,2018-12-09,eredivisie,fc emmen,feyenoord,home_win,7.20,0.1640,1.180800
91,2018-12-09,eredivisie,nac breda,vitesse,home_win,5.50,0.2398,1.318900
169,2018-12-09,eredivisie,fc emmen,feyenoord,tie,5.20,0.2278,1.184560
1,2018-12-15,eredivisie,pec zwolle,nac breda,away_win,5.50,0.2375,1.306250
14,2018-12-09,la_liga,real betis,rayo vallecano,away_win,6.20,0.2001,1.240620


## Congratulations. You will be very rich, sir.
Now save the analysis results.

In [11]:
# first add the date of today
from datetime import datetime
now = datetime.now()
dfBets['date_of_bet'] = '%s-%s-%s' % (now.day, now.month, now.year)

# write to Book.csv - caution!
#dfBets.to_csv('./Book.csv', index=False, float_format='%.3f')

# append to Book.csv
dfBets.to_csv('./Book.csv', index=False, float_format='%.3f', mode='a', header=False)